In [8]:
import pandas as pd

salary_df = pd.read_excel('./salary_data_states.xlsx')
salary_df.head()

,CASE_NUMBER,CASE_STATUS,CASE_RECEIVED_DATE,DECISION_DATE,EMPLOYER_NAME,PREVAILING_WAGE_SUBMITTED,PREVAILING_WAGE_SUBMITTED_UNIT,PAID_WAGE_SUBMITTED,PAID_WAGE_SUBMITTED_UNIT,JOB_TITLE,...,PREVAILING_WAGE_SOC_TITLE,WORK_STATE,WORK_STATE_ABBREVIATION,WORK_POSTAL_CODE,FULL_TIME_POSITION_Y_N,VISA_CLASS,PREVAILING_WAGE_PER_YEAR,PAID_WAGE_PER_YEAR,JOB_TITLE_SUBGROUP,order
0,I-200-14073-248840,denied,3/14/2014,3/21/2014,"ADVANCED TECHNOLOGY GROUP USA, INC.",6217100.0,year,62171.0,year,SOFTWARE ENGINEER,...,"Software Developers, Applications",Illinois,IL,NaN,y,H-1B,NaN,62171.0,software engineer,1
1,A-15061-55212,denied,3/19/2015,3/19/2015,SAN FRANCISCO STATE UNIVERSITY,5067600.0,year,91440.0,year,Assistant Professor of Marketing,...,"Business Teachers, Postsecondary",California,CA,94132.0,NaN,greencard,NaN,91440.0,assistant professor,2
2,I-200-13256-001092,denied,9/13/2013,9/23/2013,CAROUSEL SCHOOL,4947000.0,year,49470.0,year,SPECIAL EDUCATION TEACHER,...,"Special Education Teachers, Kindergarten and E...",California,CA,NaN,y,H-1B,NaN,49470.0,teacher,3
3,I-200-14087-353657,denied,3/28/2014,4/7/2014,HARLINGEN CONSOLIDATED INDEPENDENT SCHOOL DIST...,251052.0,month,43800.0,year,SCIENCE TEACHER,...,"Biological Science Teachers, Postsecondary",Texas,TX,NaN,y,H-1B,NaN,43800.0,teacher,4
4,I-203-14259-128844,denied,9/16/2014,9/23/2014,SIGNAL SCIENCES CORPORATION,84573.0,bi-weekly,170000.0,year,SENIOR SOFTWARE ENGINEER,...,"Software Developers, Systems Software",Oregon,OR,NaN,y,E-3 Australian,NaN,170000.0,software engineer,5


In [13]:
salary_df.describe()

,PREVAILING_WAGE_SUBMITTED,PAID_WAGE_SUBMITTED,EXPERIENCE_REQUIRED_NUM_MONTHS,PREVAILING_WAGE_PER_YEAR,PAID_WAGE_PER_YEAR,order
count,1.672780e+05,1.672780e+05,4965.000000,167210.000000,1.672780e+05,167278.000000
mean,7.115752e+04,8.164186e+04,34.692044,74274.868236,8.553277e+04,83714.716305
std,3.874624e+04,4.147703e+04,22.317783,25356.245893,3.873847e+04,48300.236431
min,5.050000e+00,5.050000e+00,0.000000,10504.000000,1.050000e+04,1.000000
25%,5.447500e+04,6.100000e+04,12.000000,56880.000000,6.300000e+04,41901.250000
50%,6.970100e+04,7.700000e+04,24.000000,70928.000000,7.860000e+04,83722.500000
75%,8.993700e+04,1.000000e+05,60.000000,90459.000000,1.000060e+05,125541.750000
max,6.217100e+06,2.500000e+06,144.000000,320000.000000,2.500000e+06,167361.000000


In [ ]:
import seaborn as sns
sns.histplot(a= salary_df['PREVAILING_WAGE_PER_YEAR']);
sns.histplot(b = salary_df['EXPERIENCE_REQUIRED_NUM_MONTHS']);
sns.histplot(c = salary_df['WORK_STATE']);
sns.histplot(d= salary_df['JOB_TITLE']);
sns.histplot(e = salary_df['EMPLOYER_NAME']);

             

In [ ]:
# I. Do specific sub-types of data-related jobs have higher or lower salaries than others ?

# 1a - Which companies have the highest salries for those subtypes ?

# 1b - Changes with location of the job ?

# 1c - Will the answer change if I take the standard of living into account ?

In [ ]:
# II. What states (of those I am willing to move to) have the highest paying data-related salaries ?
 
# 2a - Differences between job subcategories ?

# 2b - Which companies have the highest salaries of those sub-types ?

# 2c - Will the answer change if I take the standard of living into account ?

In [ ]:
# III. How do offered salaries compare to the prevailing wage ?

# 3a - Are there job sub-categories that tend to get over- or under- paid ?

# 3b - Are there companies that tend to over- or under- pay ?

# 3c - Will the answer change if I take the standard of living into account ?

In [ ]:
# IV. 

In [ ]:
# V.

In [ ]:
# VI.